In [31]:
#default_exp filter

In [3]:
#hide
import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Filtragem
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [4]:
#export
import os
from pathlib import Path
import json
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
from anateldb.query import *
from anateldb.constants import console, APP_ANALISE
from fastcore.test import *
from fastcore.script import call_parse, Param, store_true
from pyarrow import ArrowInvalid
from geopy.distance import geodesic 
from rich import print

In [5]:
#export
def bump_version(version, part=2):
    version = version.split('.')
    version[part] = str(int(version[part]) + 1)
    for i in range(part+1, 3): version[i] = '0'
    return '.'.join(version)

In [6]:
#exporti
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    @param filename: File path or existing ExcelWriter
                     (Example: '/path/to/file.xlsx')
    @param df: DataFrame to save to workbook
    @param sheet_name: Name of sheet which will contain DataFrame.
                       (default: 'Sheet1')
    @param startrow: upper left cell row to dump data frame.
                     Per default (startrow=None) calculate the last row
                     in the existing DF and write to the next row...
    @param truncate_sheet: truncate (remove and recreate) [sheet_name]
                           before writing DataFrame to Excel file
    @param to_excel_kwargs: arguments which will be passed to `DataFrame.to_excel()`
                            [can be a dictionary]
    @return: None

    Usage examples:

    >>> append_df_to_excel('d:/temp/test.xlsx', df)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, header=None, index=False)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, sheet_name='Sheet2',
                           index=False)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, sheet_name='Sheet2', 
                           index=False, startrow=25)

    (c) [MaxU](https://stackoverflow.com/users/5741205/maxu?tab=profile)
    """
    # Excel file doesn't exist - saving and exiting
    if not Path(filename).is_file():
        df.to_excel(
            filename,
            sheet_name=sheet_name, 
            startrow=startrow if startrow is not None else 0, 
            **to_excel_kwargs)
        return
    
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a', if_sheet_exists='replace')

    # try to open an existing workbook
    writer.book = load_workbook(filename)
    
    # get the last row in the existing Excel sheet
    # if it was not specified explicitly
    if startrow is None and sheet_name in writer.book.sheetnames:
        startrow = writer.book[sheet_name].max_row

    # truncate sheet
    if truncate_sheet and sheet_name in writer.book.sheetnames:
        # index of [sheet_name] sheet
        idx = writer.book.sheetnames.index(sheet_name)
        # remove [sheet_name]
        writer.book.remove(writer.book.worksheets[idx])
        # create an empty sheet [sheet_name] using old index
        writer.book.create_sheet(sheet_name, idx)
    
    # copy existing sheets
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [7]:
#export
@call_parse
def formatar_db(
    path: Param("Pasta onde salvar os arquivos", str),
    up_stel: Param("Atualizar a base do Stel", store_true) = False, 
    up_radcom: Param("Atualizar a base do Radcom", store_true) = False,
    up_mosaico: Param("Atualizar a base do Mosaico", store_true) = False,
    up_icao: Param("Atualizar a base do ICAO", store_true) = False,
    up_pmec: Param("Atualizar a base do PMEC", store_true) = False,
    up_geo: Param("Atualizar a base do Geo", store_true) = False,
) -> None:
    dest = Path(path)
    dest.mkdir(parents=True, exist_ok=True)
    time = datetime.today().strftime("%d/%m/%Y %H:%M:%S")
    console.print(":scroll:[green]Lendo as bases de dados...")
    rd = read_base(path, up_stel, up_radcom, up_mosaico, up_icao)
    rd['Validade_RF'] = rd.Validade_RF.astype('string').fillna('')
    rd['Data_Ato'] = rd.Data_Ato.astype('string').fillna('')
    rd['Status'] = rd.Status.astype('string')
    rd['Classe'] = rd.Classe.astype('string')
    rd.loc[rd['Status'] != '', 'Status'] = rd.loc[rd['Status'] != '', 'Status'] + ", " \
        + rd.loc[rd['Status'] != '', 'Classe']
    rd.loc[rd['Status'].isna(), 'Status'] = rd.loc[rd['Status'].isna(), 'Num_Serviço'].astype('string')

    rd["Descrição"] = (
        '[' + rd.Fonte.astype('string') + '] ' + 
        rd.Status.astype('string').fillna('-')
        + ", "
        + rd.Entidade.astype('string').fillna('-').str.title()
        + " ("
        + rd.Fistel.astype('string').fillna('-')
        + ", "
        + rd["Número_da_Estação"].astype('string').fillna('-')
        + "), "
        + rd.Município.astype('string').fillna('-')
        + "/"
        + rd.UF.astype('string').fillna('-')
    )

    export_columns = [
        "Frequência",
        "Latitude",
        "Longitude",
        "Descrição",
        "Num_Serviço",
        "Número_da_Estação",
        "Classe_Emissão",
        "Largura_Emissão",
    ]
    rd = rd.loc[:, export_columns]
    rd.columns = APP_ANALISE
    rd  = merge_aero(rd, read_aero(path, up_icao, up_pmec, up_geo))
    rd = df_optimize(rd, exclude=['Frequency'])
    console.print(":card_file_box:[green]Salvando os arquivos...")
    d = json.loads((dest / 'VersionFile.json').read_text())
    try:
        cache = pd.read_feather(f"{dest}/AnatelDB.fth")
    except (ArrowInvalid, FileNotFoundError):
        cache = pd.DataFrame()
    
    if not rd.equals(cache):
        console.print(":new: [green] A base de dados mudou desde a última atualização! Salvando o novo arquivo e atualizando a versão")
        date = pd.DataFrame(columns=[time])
        try:
            rd.to_feather(Path(f"{dest}/AnatelDB.fth").open('bw'))
        except ArrowInvalid:
            Path(f"{dest}/AnatelDB.fth").unlink()
        with pd.ExcelWriter(f"{dest}/AnatelDB.xlsx", engine='xlsxwriter') as workbook:
            date.to_excel(workbook, sheet_name="ExtractDate", index=False)
            rd.to_excel(workbook, sheet_name="DataBase", index=False)
        d['anateldb']['Version'] = bump_version(d['anateldb']['Version'])
    else:
        console.print(":recycle: [green] A base de dados não mudou desde a última atualização, a versão não será atualizada, somente a data de verificação")


    console.print("Sucesso :zap:")    
    d['anateldb']['ReleaseDate'] = datetime.today().strftime('%d/%m/%Y')
    json.dump(d, (dest / 'VersionFile.json').open('w'))

In [8]:
pasta = Path('c:/Users/rsilva/AnatelDatabase')

In [9]:
if not pasta.exists():
    import subprocess
    result = subprocess.run(['git', 'clone', 'https://github.com/ronaldokun/AnatelDatabase.git'])
    pasta = Path.cwd() / 'AnatelDatabase'

In [39]:
# formatar_db(pasta)

In [40]:
#df = pd.read_feather(f'{pasta}/AnatelDB.fth') #, engine='openpyxl', sheet_name='DataBase')

In [41]:
#df.tail()

In [42]:
#df.info()

In [145]:
icao = read_icao(pasta)
pmec = read_pmec(pasta)
geo = read_geo(pasta)
icao['Description'] = icao.Description.astype('string')
pmec['Description'] = pmec.Description.astype('string')
geo['Description'] = geo.Description.astype('string')

In [146]:
print(icao.shape, pmec.shape, geo.shape)

(2632, 6)
(971, 6)
(285, 6)

In [147]:
cols = ['Frequency', 'Station']
a = icao[icao.Station != -1].reset_index(drop=True)
b = pmec[pmec.Station != -1].reset_index(drop=True)
c = geo[geo.Station != -1].reset_index(drop=True)

In [148]:
df = merge_df(a, b, cols, how='outer')
df = merge_df(df, c, cols, how='outer')
df.drop_duplicates(inplace=True, keep='first')
df

,Frequency,Station,Description,Latitude,Longitude,Service
0,109.100,696597080,"[ICAO] ILS/DME, JUIZ DE FORA",-21.783333,-43.383335,108.0
1,109.300,1008242052,"[ICAO] ILS, RIO DE JANEIRO INTL.",-22.799999,-43.216667,108.0
2,110.300,1000822769,"[ICAO] ILS, CAMPINAS VIRACOPOS, SP | [AISW] SB...",-23.017500,-47.117500,108.0
3,110.700,1000823269,"[ICAO] ILS, SAO PAULO GUARULHOS INTL. | [AISW]...",-23.425468,-46.464542,108.0
4,111.100,1000823242,"[ICAO] ILS, SAO PAULO GUARULHOS INTL. | [AISW]...",-23.436501,-46.486748,108.0
...,...,...,...,...,...,...
1606,1193.000,1000822904,"[DOC] SBME-RDONAV, VOR/DME MCA, MACAE (Ground-...",-22.344475,-41.769001,108.0
1607,1195.000,1000823692,"[DOC] SBUL-RDONAV, VOR/DME ULD, TEN CEL AV CÉS...",-18.876638,-48.221313,108.0
1608,1198.000,1000823781,"[DOC] SBVT-RDONAV, VOR/DME VRI, Eurico de Agui...",-20.260166,-40.285000,108.0
1609,0.355,690266120,[AISG] NDB - URUCU OPR PETROBRAS,-4.886500,-65.349831,108.0


In [25]:
def merge_diff(df):
    """Merge two dataframes"""
    x = df.Description_x.notna()
    y = df.Description_y.notna()
    df.loc[x & ~y, 'Description'] = df.loc[x & ~y, 'Description_x']
    df.loc[~x & y, 'Description'] = df.loc[~x & y, 'Description_y']
    df.loc[x & ~y, 'Latitude'] = df.loc[x & ~y, 'Latitude_x']
    df.loc[x & ~y, 'Longitude'] = df.loc[x & ~y, 'Longitude_x']
    df.loc[~x & y, 'Latitude'] = df.loc[~x & y, 'Latitude_y']
    df.loc[~x & y, 'Longitude'] = df.loc[~x & y, 'Longitude_y']
    df.loc[x, 'Service'] = df.loc[x, 'Service_x']
    df.loc[~x & y, 'Service'] = df.loc[~x & y, 'Service_y']
    return df.loc[:, [c for c in df.columns if '_' not in c]]


In [149]:
cols = ['Frequency']
a = icao[icao.Station == -1].drop(columns=['Service', 'Station']).reset_index(drop=True)
b = pmec[pmec.Station == -1].drop(columns=['Service', 'Station']).reset_index(drop=True)
c = geo[geo.Station == -1].drop(columns=['Service', 'Station']).reset_index(drop=True)

In [150]:
print(a.shape, b.shape, c.shape)

(1148, 4)
(600, 4)
(213, 4)

In [151]:
abc = pd.merge(a,b, on=cols, how='outer').merge(c, on=cols, how='outer').drop_duplicates().reset_index(drop=True)
abc

,Frequency,Latitude_x,Longitude_x,Description_x,Latitude_y,Longitude_y,Description_y,Latitude,Longitude,Description
0,109.1,-25.600000,-54.466667,"[ICAO] ILS, FOZ DO IGUACU CATARATAS",-25.602833,-54.478500,"[AISW] SBFI-RDONAV, ILS/DME 14 IFI, Cataratas",NaN,NaN,<NA>
1,109.3,-1.366667,-48.466667,"[ICAO] ILS/DME, BELEM VAL DE CAES",-2.578333,-44.228333,"[AISW] SBSL-RDONAV, ILS/DME 06 ISL, Marechal C...",NaN,NaN,<NA>
2,109.3,-1.366667,-48.466667,"[ICAO] ILS/DME, BELEM VAL DE CAES",-23.637667,-46.648834,"[AISW] SBSP-RDONAV, ILS/DME 17R ISP, Congonhas",NaN,NaN,<NA>
3,109.3,-1.366667,-48.466667,"[ICAO] ILS/DME, BELEM VAL DE CAES",-20.267000,-40.292835,"[AISW] SBVT-RDONAV, ILS/DME 24 IVI, Eurico de ...",NaN,NaN,<NA>
4,109.3,-1.366667,-48.466667,"[ICAO] ILS/DME, BELEM VAL DE CAES",-9.861167,-67.881668,"[AISW] SBRB-RDONAV, ILS/DME 06 IRB, Plácido de...",NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...
4270,116.1,NaN,NaN,<NA>,NaN,NaN,<NA>,-27.135693,-52.662968,[AISG] VOR - CHAPECÓ OPR CITY HALL CH 108X
4271,967.0,NaN,NaN,<NA>,NaN,NaN,<NA>,-2.578333,-44.228333,[AISG] DME - SÃO LUIZ 6X
4272,985.0,NaN,NaN,<NA>,NaN,NaN,<NA>,-21.700470,-41.307671,[AISG] DME - CAMPOS 24X
4273,1182.0,NaN,NaN,<NA>,NaN,NaN,<NA>,-22.967175,-42.890945,[AISG] DME - MARICÁ 95X


In [152]:
x = abc.Description_x.notna()
y = abc.Description_y.notna()
z = abc.Description.notna()
new = pd.DataFrame(columns=['Frequency', 'Latitude', 'Longitude', 'Description'])

In [153]:
for c,n in zip([(x & ~y & ~z), (~x & y & ~z), (~x & ~y & z)], ['_x', '_y', '']):
    temp = abc.loc[c, ['Frequency', f'Latitude{n}', f'Longitude{n}', f'Description{n}']]
    temp.columns = ['Frequency', 'Latitude', 'Longitude', 'Description']
    new = pd.concat([new, temp], ignore_index=True)
new.drop_duplicates(inplace=True, keep='first')
new

,Frequency,Latitude,Longitude,Description
0,112.2,-22.35,-49.049999,"[ICAO] VOR/DME, BAURU"
1,112.2,-14.8,-39.016666,"[ICAO] VOR/DME, ILHEUS"
2,112.3,-12.9,-38.400002,"[ICAO] VOR, ITAPARICA"
3,112.3,-5.066667,-42.816666,"[ICAO] VOR/DME, TERESINA"
4,112.5,-5.8,-61.283333,"[ICAO] VOR, MANICORE"
...,...,...,...,...
976,116.1,-27.135693,-52.662968,[AISG] VOR - CHAPECÓ OPR CITY HALL CH 108X
977,967.0,-2.578333,-44.228333,[AISG] DME - SÃO LUIZ 6X
978,985.0,-21.70047,-41.307671,[AISG] DME - CAMPOS 24X
979,1182.0,-22.967175,-42.890945,[AISG] DME - MARICÁ 95X


In [154]:
for c, (left, right) in zip([(x & y & ~z), (x & ~y & z), (~x & y & z)], [('_x','_y'), ('_x',''), ('_y','')]):
    for row in abc[c].itertuples():
        d = geodesic((getattr(row, f'Latitude{left}'), getattr(row, f'Longitude{left}')), (getattr(row, f'Latitude{right}'), getattr(row, f'Longitude{right}'))).m
        if d < 500:
            f = row.Frequency
            lat = (getattr(row, f'Latitude{left}') + getattr(row, f'Latitude{right}')) / 2
            lon = (getattr(row, f'Longitude{left}') + getattr(row, f'Longitude{right}')) / 2
            d = (getattr(row, f'Description{left}') + ' | ' + getattr(row, f'Description{right}'))
            new = pd.concat([new, pd.DataFrame([[f, lat, lon, d]], columns=['Frequency', 'Latitude', 'Longitude', 'Description'])], ignore_index=True)
        else:
            l = abc.loc[row.Index, ['Frequency', f'Latitude{left}', f'Longitude{left}', f'Description{left}']].to_frame().T
            l.columns = new.columns
            r = abc.loc[row.Index, ['Frequency', f'Latitude{right}', f'Longitude{right}', f'Description{right}']].to_frame().T
            r.columns = new.columns
            new = pd.concat([new, l, r], ignore_index=True)
new.drop_duplicates(inplace=True, keep='first')
new            

,Frequency,Latitude,Longitude,Description
0,112.2,-22.35,-49.049999,"[ICAO] VOR/DME, BAURU"
1,112.2,-14.8,-39.016666,"[ICAO] VOR/DME, ILHEUS"
2,112.3,-12.9,-38.400002,"[ICAO] VOR, ITAPARICA"
3,112.3,-5.066667,-42.816666,"[ICAO] VOR/DME, TERESINA"
4,112.5,-5.8,-61.283333,"[ICAO] VOR, MANICORE"
...,...,...,...,...
2723,1192.0,-21.984501,-47.344501,"[DOC] SBYS-RDONAV, VOR/DME PIR, CAMPO FONTENEL..."
2724,1192.0,-3.04003,-60.054737,[AISG] DME - MANAUS 105X
2725,1198.0,-9.87599,-67.905357,"[DOC] SBRB-RDONAV, VOR/DME RCO, Plácido de Cas..."
2726,1200.0,-0.150306,-66.990536,"[DOC] SBUA-RDONAV, VOR/DME GBR, São Gabriel da..."


In [155]:
for row in abc[x & y & z].itertuples():
    d1 = geodesic((getattr(row, 'Latitude_x'), getattr(row, 'Longitude_x')), (getattr(row, 'Latitude_y'), getattr(row, 'Longitude_y'))).m
    d2 = geodesic((getattr(row, 'Latitude_x'), getattr(row, 'Longitude_x')), (getattr(row, 'Latitude'), getattr(row, 'Longitude'))).m
    d3 = geodesic((getattr(row, 'Latitude_y'), getattr(row, 'Longitude_y')), (getattr(row, 'Latitude'), getattr(row, 'Longitude'))).m
    if d1 < 500 and d2 < 500 and d3 < 500:
        f = row.Frequency
        lat = (row.Latitude_x + row.Latitude_y + row.Latitude) / 3
        lon = (row.Longitude_x + row.Longitude_y + row.Longitude) / 3
        d = ' | '.join([row.Description_x, row.Description_y, row.Description])
        new = pd.concat([new, pd.DataFrame([[f, lat, lon, d]], columns=['Frequency', 'Latitude', 'Longitude', 'Description'])], ignore_index=True)
    elif d1 < 500 and d2 < 500:
        f = row.Frequency
        lat = (row.Latitude_x + row.Latitude_y) / 2
        lon = (row.Longitude_x + row.Longitude_y) / 2
        d = ' | '.join([row.Description_x, row.Description_y])
        new = pd.concat([new, pd.DataFrame([[f, lat, lon, d]], columns=['Frequency', 'Latitude', 'Longitude', 'Description'])], ignore_index=True)
    elif d1 < 500 and d3 < 500:
        f = row.Frequency
        lat = (row.Latitude_x + row.Latitude) / 2
        lon = (row.Longitude_x + row.Longitude) / 2
        d = ' | '.join([row.Description_x, row.Description])
        new = pd.concat([new, pd.DataFrame([[f, lat, lon, d]], columns=['Frequency', 'Latitude', 'Longitude', 'Description'])], ignore_index=True)
    elif d2 < 500 and d3 < 500:
        f = row.Frequency
        lat = (row.Latitude_y + row.Latitude) / 2
        lon = (row.Longitude_y + row.Longitude) / 2
        d = ' | '.join([row.Description_y, row.Description])
        new = pd.concat([new, pd.DataFrame([[f, lat, lon, d]], columns=['Frequency', 'Latitude', 'Longitude', 'Description'])], ignore_index=True)
    else:
        l = abc.loc[row.Index, ['Frequency', 'Latitude_x', 'Longitude_x', 'Description_x']].to_frame().T
        l.columns = new.columns
        r = abc.loc[row.Index, ['Frequency', 'Latitude_y', 'Longitude_y', 'Description_y']].to_frame().T
        r.columns = new.columns
        s = abc.loc[row.Index, ['Frequency', 'Latitude', 'Longitude', 'Description']].to_frame().T
        s.columns = new.columns
        new = pd.concat([new, l, r, s], ignore_index=True)
new.drop_duplicates(inplace=True, keep='first')
new

,Frequency,Latitude,Longitude,Description
0,112.2,-22.35,-49.049999,"[ICAO] VOR/DME, BAURU"
1,112.2,-14.8,-39.016666,"[ICAO] VOR/DME, ILHEUS"
2,112.3,-12.9,-38.400002,"[ICAO] VOR, ITAPARICA"
3,112.3,-5.066667,-42.816666,"[ICAO] VOR/DME, TERESINA"
4,112.5,-5.8,-61.283333,"[ICAO] VOR, MANICORE"
...,...,...,...,...
8605,1211.0,-19.833334,-44.0,"[DOC] VOR/DME, BELO HORIZONTE PAMPULHA (Ground..."
8606,1211.0,-19.835833,-44.003502,"[DOC] SBBH-RDONAV, VOR/DME BHZ a. SECT 100/11..."
8607,1211.0,4.693,-61.028831,[AISG] DME - LA DIVINA PASTORA 124X
8610,1211.0,-25.778656,-49.763241,[AISG] DME - LAPA-PR 124X


In [157]:
new.to_excel(pasta / 'aeronautica.xlsx', index=False)

In [48]:
import hdf5storage as hdf

In [60]:
db = hdf.loadmat(f'{pasta}/AnatelDB.mat')

In [59]:
db = hdf.read(path='/c/Users/rsilva/db/', filename='AnatelDB.mat')

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'AnatelDB.mat', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [16]:
# import seaborn as sns
# import matplotlib.pyplot as plt


# ax = sns.countplot(x="Num_Serviço", data=base)

# plt.rcParams["figure.figsize"] = [30, 20]
# plt.rcParams["figure.autolayout"] = True

# for p in ax.patches:
#     ax.annotate('{:.1f}'.format(int(p.get_height())), (p.get_x()+0.05, p.get_height()+0.02))
    
# plt.title(f"Total de Entidades: {base.shape[0]}")
# plt.xlabel("Código Serviço")

# plt.savefig("Stats.png")

In [11]:
base = read_base(pasta)

In [12]:
base.head()

,Frequência,Num_Serviço,Status,Classe,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,Validade_RF,Num_Ato,Data_Ato,Fonte,Classe_Emissão,Largura_Emissão
0,0.0280,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557670,Nova Iguaçu,RJ,-22.662777,-43.476387,2033-08-17,-1,,STEL,J9E,8K00
1,0.0285,19,L,OP,COMPANHIA DE GERAÇÃO E TRANSMISSÃO DE ENERGIA ...,50420217282,1494686,Joinville,SC,-26.292500,-48.887222,2025-08-31,-1,,STEL,R3E,2K50
2,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,859966,Araporã,MG,-18.410000,-49.099998,2033-08-17,-1,,STEL,J3E,1K00
3,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,859753,Campinas,SP,-22.774166,-47.004444,2033-08-17,-1,,STEL,J3E,1K00
4,0.0300,19,L,OP,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557823,São Paulo,SP,-23.441668,-46.590832,2033-08-17,-1,,STEL,J3E,1K00


In [13]:
subset = ['Frequência', 'Num_Serviço', 'Status', 'Classe', 'Fistel', 'Número_da_Estação', 'Latitude', 'Longitude', 'Classe_Emissão', 'Largura_Emissão']
# double = base.duplicated(subset=subset, keep=False)
# base[double]

base.drop_duplicates(subset, keep='first').shape

(881692, 17)

In [105]:
subset = base.columns[:-2]
# double = base.duplicated(subset=subset, keep='first')
# base[double]

base.drop_duplicates(base.columns[:-2], keep='first').reset_index(drop=True)
# base.shape

,Frequência,Num_Serviço,Status,Classe,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,Validade_RF,Num_Ato,Data_Ato,Fonte,Classe_Emissão,Largura_Emissão
26,0.038,19,L,OP,LIGHT SERVICOS DE ELETRICIDADE S A,01030063621,756830,Piraí,RJ,-22.629444,-43.895832,2028-12-12,-1,,STEL,N0N,1K00
33,0.040,19,L,OP,LIGHT SERVICOS DE ELETRICIDADE S A,01030063621,758124,Petrópolis,RJ,-22.433332,-43.166668,2028-12-12,-1,,STEL,A3E,5K00
37,0.040,19,L,OP,LIGHT SERVICOS DE ELETRICIDADE S A,01030063621,758051,Carmo,RJ,-21.840000,-42.570000,2028-12-12,-1,,STEL,N0N,1K00
58,0.044,19,L,OP,LIGHT SERVICOS DE ELETRICIDADE S A,01030063621,814180,Volta Redonda,RJ,-22.513611,-44.089722,2028-12-12,-1,,STEL,A3E,5K00
62,0.045,19,L,OP,LIGHT SERVICOS DE ELETRICIDADE S A,01030063621,749680,Rio de Janeiro,RJ,-22.966667,-43.233334,2028-12-12,-1,,STEL,A3E,5K00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883461,82125.000,19,L,FX,TIM S A,50417425295,1007302558,Ribeirão das Neves,MG,-19.768250,-44.082527,2032-08-31,-1,,STEL,Q7W,62M5
883467,82125.000,19,L,FX,TIM S A,50417425295,1005319658,Uberlândia,MG,-18.942499,-48.300556,2032-08-31,-1,,STEL,Q7W,62M5
883481,82125.000,19,L,FX,TIM S A,50417425295,1005325984,Araxá,MG,-19.582222,-46.953888,2032-08-31,-1,,STEL,Q7W,62M5
883514,82125.000,19,L,FX,TIM S A,50417425295,1008765390,Aparecida de Goiânia,GO,-16.805166,-49.329556,2032-08-31,-1,,STEL,Q7W,62M5


In [14]:
base[base.Número_da_Estação == 1005304065]

,Frequência,Num_Serviço,Status,Classe,Entidade,Fistel,Número_da_Estação,Município,UF,Latitude,Longitude,Validade_RF,Num_Ato,Data_Ato,Fonte,Classe_Emissão,Largura_Emissão
744796,8118.32,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,D7W,29M7
825221,19343.00,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,D7W,55M0
831552,19453.00,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,D7W,55M0
865366,23275.00,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,D7W,56M0
881466,82125.00,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,G7W,2G00
881467,82125.00,19,L,FX,TIM S A,50417425295,1005304065,Sabará,MG,-19.897028,-43.807056,2032-08-31,-1,,STEL,Q7W,62M5


In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted constants.ipynb.
Converted filter.ipynb.
Converted index.ipynb.
Converted queries.ipynb.
